ДЕКОМПРЕССИЯ

In [234]:
romName = r"d:\DEV\CadEditor\CadEditor\Felix the Cat (U) [!].nes"
with open(romName, "rb") as f:
    d = f.read()
d = map(ord, d)

In [235]:
def readWord(d, addr):
    return d[addr+1]<<8 | d[addr]

In [236]:
def addPtrToLinesPtr(addr):
    return addr + 0x10010

def addPtrToCompress(addr):
    return addr + 0x10010

def addPtrToLines(addr):
    if addr < 0x8000:
        addr += 0x8000
    if addr > 0xE000:
        addr -= 0x4000
    return addr + 0x10010

In [237]:
linesPtrsAddr = addPtrToLinesPtr(readWord(d,0x27e))
print(hex(linesPtrsAddr))
compressAddr = addPtrToCompress(readWord(d,0x280))
print(hex(compressAddr))

0x1b93f
0x1b799


In [238]:
def readCompress(d, addr):
    ans = []
    for c in xrange(128):
        ans.append((d[addr], d[addr+1]))
        addr += 2
    return ans

In [239]:
compressedArr = readCompress(d, compressAddr)

In [240]:
def decompressLine(d, addr, LINE_HEIGHT = 24):
    global maxCV
    ans = []
    while len(ans) < LINE_HEIGHT:
        v = d[addr]
        if v < 0x80:
            ans.append(v)
        else:
            cv = v & 0x7F
            repeatCount, repeatTile = compressedArr[cv]
            ans.extend([repeatTile]*repeatCount)
        addr += 1
    return ans
    #return ans[:LINE_HEIGHT]

In [241]:
def decompressScreen():
    screen = []
    #actually, it's 3 separate configs for level 1-1, 1-2, 1-3, but it has similar pointers and
    # lie together, so we simple read x3 times to get info of whole level
    WIDTH = 256*3
    HEIGHT = 24
    curLinePtrsAddr = linesPtrsAddr
    #lineAddrs = []
    for l in xrange(WIDTH):
        #print "CUR_LINE_PTR", hex(curLinePtrsAddr)
        lineAddr = addPtrToLines(readWord(d, curLinePtrsAddr))
        #lineAddrs.append(lineAddr)
        #print "LINE_ADDR", hex(readWord(d, curLinePtrsAddr))
        vals = decompressLine(d, lineAddr, HEIGHT)
        #print vals
        assert(len(vals)== HEIGHT)
        screen.extend(vals)
        curLinePtrsAddr += 2
    #print hex(min(lineAddrs))
    #print hex(max(lineAddrs))
    return screen

In [242]:
screen = decompressScreen()

In [243]:
dumpName = r"d:\DEV\CadEditor\CadEditor\settings_felix_the_cat/dump1.bin"
with open(dumpName, "wb") as f:
    v = "".join(map(chr, screen))
    f.write(v)

КОМПРЕССИЯ

In [258]:
dumpName = r"d:\DEV\CadEditor\CadEditor\settings_felix_the_cat/dump1.bin"
with open(dumpName, "rb") as f:
    d = f.read()
    
screen = map(ord, d)

In [259]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [260]:
from itertools import groupby
def packRepeat(repeat, compressedArr):
    tileNo, tileCount = repeat
    if tileCount <= 0:
        return []
    if tileCount == 1:
        return [tileNo]
    #---
    maxInd, maxCount = -1, -1
    for (ind,(c,t)) in enumerate(compressedArr):
        if t == tileNo and c <= tileCount:
            if c > maxCount:
                maxCount = c
                maxInd = ind
    if maxInd != -1:
        return [0x80 | maxInd ] + packRepeat((tileNo, tileCount - maxCount), compressedArr)
    #---
    return [tileNo] + packRepeat((tileNo, tileCount-1), compressedArr)

def compressLine(l, compressedArr):
    repeats = [list(g) for _, g in groupby(l)]
    repeats = [(g[0],len(g)) for g in repeats]
    ans = []
    for r in repeats:
        ans.extend(packRepeat(r, compressedArr))
    return ans


In [261]:
def compressScreen(screen):
    LINE_LEN = 24
    curAddr = 0xA8ED
    prevLinesAddrs = {}
    linesAddrs = []
    linesArray = []
    for line in chunks(screen, LINE_LEN):
        cline = compressLine(line, compressedArr)
        clineTupple = tuple(cline)
        if clineTupple in prevLinesAddrs:
            linesAddrs.append(prevLinesAddrs[clineTupple])
            continue
        prevLinesAddrs[clineTupple] = curAddr
        linesAddrs.append(curAddr)
        linesArray.extend(cline)
        curAddr += len(cline)
    return linesAddrs, linesArray

linesAddrs, linesArray = compressScreen(screen)

ПАТЧИНГ РОМА, ВОЗМОЖНО ОШИБКИ!

In [262]:
def wordToBytes(word):
    return word&0xFF, word>>8

In [263]:
romPatchLinesArrayAddr = 0x1B93F
romPatchDataArrayAddr  = 0x1A8FD
dataLines = []
for addr in linesAddrs:
    dataLines.extend(wordToBytes(addr))
    
romName = r"d:\DEV\CadEditor\CadEditor\Felix the Cat (U) [!].nes"
romName2 = r"d:\DEV\CadEditor\CadEditor\Felix the Cat (U) [!]-2.nes"
with open(romName, "rb") as f:
    d = f.read()
    d = map(ord, d)
#write lines pointers
for x in xrange(len(dataLines)):
    #we must save some bits it hi bytes addr, because they used to encoding objects
    if x%2 == 1:
        v = d[romPatchLinesArrayAddr+x]
        v = (v & 0xC0) | (dataLines[x] & 0x3F)
        d[romPatchLinesArrayAddr+x] = v
    else:
        d[romPatchLinesArrayAddr+x] = dataLines[x]
#write lines data
d[romPatchDataArrayAddr:romPatchDataArrayAddr+len(linesArray)] = linesArray
with open(romName2, "wb") as f:
    d =  "".join(map(chr, d))
    f.write(d)